In [1]:

##########################################################
# Image Proccessing with Deep Learning
# DLIP Final 2024 Submission
#
# Date: 2024-5-28
# 
# Author: Jin Kwak
#
# ID: 21900031
#
##########################################################

# Import Module
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt



##########################################################
## Part 0:  GPU setting
##########################################################

# Select GPU or CPU for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:


##########################################################
## Part 1:  Prepare Dataset 
##########################################################

# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../Exercise/data/hymenoptera_data"
input_size = 299

# Data augmentation and normalization for training
# Just normalization for validation
# Normalized with ImageNet mean and variance
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

training_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform['train'])
test_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform['val'])

classes = ['ant', 'bee']
print(f"train dataset length = {len(training_data)}")
print(f"test  dataset length = {len(test_data)}")

# Batch size for training (change depending on how much memory you have)
batch_size = 8

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape} {y.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



"""
##########################################################
## Part 3:  Create Model
##########################################################

------------------------------------------------------#
* TEST Q1. Create a model architecture
------------------------------------------------------#
"""

class Stem(nn.Module):
    def __init__(self):
        super(Stem, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=0, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 80, kernel_size=1, stride=1, padding=0, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(80, 192, kernel_size=3, stride=1, padding=0, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 256, kernel_size=3, stride=2, padding=0, bias=False),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.conv_layer(x)

class InceptionResNetA(nn.Module):
    def __init__(self):
        super(InceptionResNetA, self).__init__()
        self.conv1  = nn.Conv2d(256, 32, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2  = nn.Sequential(
            nn.Conv2d(256,32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.Conv2d(32,32,kernel_size= 3, stride=1, padding=1, bias=False))
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False))
        self.conv4 = nn.Sequential(nn.Conv2d(32,256,kernel_size=1,stride=1,padding=0, bias = False))
    def forward(self, x):
        x = nn.ReLU(x)
        forward_path1 = self.conv1(x)
        forward_path2 = self.conv2(x)
        forward_path3 = self.conv3(x)

        forward_path  = torch.cat([forward_path1, forward_path2, forward_path3], 1)
        forward_path  = self.conv4(forward_path)
        x = x + forward_path
        x = nn.ReLU(x)
        return x
class ReductionA(nn.Module):
    def __init__(self):
        super(ReductionA, self).__init__()
        self.max_layer1 = nn.Sequential(
            nn.MaxPool2d(kernel_size= 3, stride= 2 , padding=0 )
        )
        self.conv_layer1 = nn.Sequential(
             nn.Conv2d(256,384, kernel_size= 3, stride=2, padding= 0 , bias= False)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(256,96,kernel_size= 1, stride=1, padding=0, bias= False),
            nn.Conv2d(96, 96,kernel_size=3, stride=1, padding=1, bias= False),
            nn.Conv2d(96,256,kernel_size=3, stride=2, padding=0, bias= False),
            nn.ReLU(inplace= True)
        )
    def forward(self,x):
        x1 = self.max_layer1(x)
        x2 = self.conv_layer1(x)
        x3 = self.conv_layer2(x)
        x_final = torch.cat([x1,x2,x3])
        return x_final
class InceptionResNetV1(nn.Module):
    def __init__(self,num_classes= 10) :
        super().__init__()
        self.InceptionResNetA = InceptionResNetA()
        self.ReductionA = ReductionA()
        self.stem = Stem()
        self.flatten = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(17*17*896, 256),
            nn.ReLU(inplace= True),
            nn.Linear(256,2),
            nn.ReLU(inplace= True),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.stem.conv_layer(x)
        x = self.InceptionResNetA.forward(x)
        x = self.ReductionA.forward(x)
        x = nn.MaxPool2d(kernel_size= 3, stride= 2)
        x = self.flatten(x)
        x = self.fc_layers(x)


model = InceptionResNetV1(num_classes=2).to(device)


from torchsummary import summary
summary(model, (3, 299, 299))



##########################################################
## Part 4:  Train Model  (DON'T MODIFY)
##########################################################
# Train Function
def train(dataloader, model, loss_fn, optimizer, device, print_interval=100):
    size = len(dataloader.dataset)
    model.train()
    batch_size = dataloader.batch_size
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % print_interval == 0:
            loss, current = loss.item(), batch * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Run Train for k epoch
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device, 15)
print("Done!")



##########################################################
##########################################################


"""
------------------------------------------------------#
* TEST Q2. Classification of given images: pretrain CNN
------------------------------------------------------#
"""

from Exercise.initialize_model import initialize_model
from Exercise.train import train
from Exercise.test import test

data_dir = "D:/DLIP/DLIP_Python/DLIP_Final_Upload_2024"

model = initialize_model('squeezenet',2,True, use_pretrained=True)

In [15]:
"""
------------------------------------------------------#
* TEST Q2. Classification of given images: pretrain CNN
------------------------------------------------------#
"""

from Exercise.initialize_model import initialize_model
from Exercise.train import train
from Exercise.test import test
import torchvision
model, input_size = initialize_model('resnet',2,True, use_pretrained=True)



C:\ProgramData\miniconda3\envs\py39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '.\\DLIP_Final_Upload_2024\\Test2_Q1.jpg'

In [21]:
transform = {
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'D:/DLIP/DLIP_Python/DLIP_Final_Upload_2024'

training_data = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform['test'])


FileNotFoundError: Couldn't find any class folder in D:/DLIP/DLIP_Python/DLIP_Final_Upload_2024\test.